In [4]:
import plotly.graph_objects as go

# Define the tree structure
tree_text = """
Test Conditions
├── CTI Electrodes
│   └── Cleaned using 0.1M H2SO4 at a scan range from 0V-1.5V at 0.1V/s
└── PCB Electrodes
    ├── Cleaning Methods
    │   ├── Solvent Cleaned
    │   │   └── H2O2:NH4OH:H2O at a 1:1:2 ratio for 30 mins, then 1:1 ratio for 30 mins
    │   ├── Mechanically Cleaned
    │   │   └── Using alumina slurry for 5 mins
    │   └── Mechanical + Solvent Cleaned
    ├── Slow Scans
    │   ├── 0V-1.5V at 0.1M
    │   └── 0V-1.5V at 0.5M
    └── Fast Scans
        ├── 0V-1.5V
        │   ├── 0.1M
        │   └── 0.5M
        ├── 0.2V-1.7V
        │   ├── 0.1M
        │   └── 0.5M
        └── 0.4V-1.9V
            ├── 0.1M
            └── 0.5M
"""

# Function to parse the tree structure
def parse_tree(text):
    lines = [line.rstrip() for line in text.splitlines() if line.strip()]
    nodes = []
    edges = []

    stack = []
    prev_indent = 0
    for line in lines:
        indent = len(line) - len(line.lstrip(' ├──│'))
        name = line.strip(' ├──│')

        nodes.append(name)
        if stack and indent > prev_indent:
            edges.append((stack[-1], name))
        while stack and indent < prev_indent:
            stack.pop()
            prev_indent -= 4
        if stack and indent == prev_indent:
            stack.pop()

        stack.append(name)
        prev_indent = indent

    return nodes, edges

nodes, edges = parse_tree(tree_text)

# Create the node positions (x, y coordinates)
def get_positions(nodes, edges):
    levels = {}
    positions = {}
    level = 0

    for node in nodes:
        positions[node] = (len(levels.get(level, [])), level)
        levels.setdefault(level, []).append(node)
        level += 1

        for edge in edges:
            if edge[0] == node:
                positions[edge[1]] = (len(levels.get(level, [])), level)
                levels.setdefault(level, []).append(edge[1])

        level -= 1

    return positions

node_positions = get_positions(nodes, edges)

# Create the figure
fig = go.Figure()

# Add nodes
for node in nodes:
    x, y = node_positions[node]
    fig.add_trace(go.Scatter(
        x=[x], y=[y],
        mode='markers+text',
        marker=dict(size=20, color='LightSkyBlue'),
        text=[node],
        textposition="top center"
    ))

# Add edges
for edge in edges:
    x0, y0 = node_positions[edge[0]]
    x1, y1 = node_positions[edge[1]]
    fig.add_trace(go.Scatter(
        x=[x0, x1], y=[y0, y1],
        mode='lines',
        line=dict(width=2, color='gray')
    ))

# Update layout
fig.update_layout(
    title="Test Conditions Tree Diagram",
    showlegend=False,
    xaxis=dict(showticklabels=False),
    yaxis=dict(showticklabels=False)
)

# Show the plot
fig.show()
